In [5]:
after = ['It’s so weird that pro-Israel people in my mentions have resorted to commenting non-stop on the size of my nose, seemingly with zero awareness about how uhhhh it is to ridicule nose size in service of a Jewish ethnostate.', 'Being a Jewish leftist is having the absolute moral necessity of opposing genocide drilled into you since birth then being told to kindly shut the fuck up by the people who did the drilling when you actually apply it', 'it still surprises me whenever I see people act shocked that there are Middle Eastern Christians like honey you do realise that it was Jesus of Nazareth not Jesus of New Jersey', 'It’s weird how people expect you to believe that Israel has Palestinians corralled into ghettos purely because of faults of their own, unrelated to the fact that if everyone in the occupied territories was enfranchised Israel would not be a majority Jewish state', 'It boggles my mind that most of you have never had the experience of watching the world mass slaughter your people, deem them dispensable, unworthy of life.   As an Arab at the age of 34, it’s all I’ve ever known.', 'When people say equal rights for Palestinians, including the right to return, would threaten Israel’s existence as a “Jewish state” you can say “what?? are you an ethnonationalist?????????” That is the starting place', 'I keep thinking how crazy it would be if an Arab country kept 2.3 million Jewish people in a cage, half children, cut off their electricity & water then indiscriminately bombed them. Beyond comprehension the anti-Muslim bigotry that makes the world applaud this as justified', 'The idea that “From the river to the sea Palestine will be free” is about expelling Jews for being Jewish instead of the fact that Israel has millions of people permanently confined in a deranged and terroristic military occupation is just denialism please be serious', 'a lot of pro-israel messaging is as though you have… no jewish friends? have never met a jewish person before? but being jewish is normal. It’s regular stuff', 'People in the West don’t realize the extent to which the carnage in Gaza is shaping and changing Arab and Muslim attitudes toward the West in general. You listen too much to token Arabs at think tanks and to UAE/Saudi propagandists.  Here is an Arab director:  “Things are not…', 'To all the nasty people weaponizing Jewish identity to center themselves as the victim while a literal holocaust is being waged on the Palestinian people: fuck you, fuck Zionism, fuck your crocodile tears. Grow the fuck up.', 'My Jewish values don’t include slaughter, collective punishment, the intentional starving of people, ethnic cleansing, genocide. My Jewish values cry out, CEASEFIRE NOW!', 'I will never forgive the British and other European powers who didn’t want to deal with Jewish refugees after the holocaust and didn’t care about the Palestinian people living on the land and set up a situation that was always going to cause violence w/o any responsibility', 'As a Jewish person, I wish I could spend this week grieving and mourning. But the Israeli government is weaponizing our grief to enact a second Nakba, killing thousands and displacing millions. I can make time to grieve while also working to stop ethnic cleansing.', 'Jewish people are not only safe in America. They are privileged.   Stop playing the “victim card.”']

before = ['Two things I periodically relearn and forget are that there are less Jewish people in the world than you’d think, and that bras are pretty expensive.', 'Whether its Democrats intervening to make sure Jewish people arent targeted on Twitter, or Elon Musk intervening to make sure murderous Russians arent targeted by Ukrainians who want to defend their own country, both sides need to mind their own business.', 'Israeli police are brutally cracking down on Eritrean refugees in Tel Aviv. The land they dispossessed Palestinians from and made them refugees.  Today we see the essence and the core of Israeli supremacy. A single ethnoreligius state for White Jewish people.', 'not to be middle eastern on the tl but i’m here freezing my ass off in my turtleneck and jacket and half of the people are vibing in their t shirts and shorts', 'I’m just a Jewish person who can’t figure out if Jesus Christ’s  middle  initial was F or H', 'I think a lot of queer white folks get very accustomed to being the most progressive person in THEIR social circles. Then they move into circles with BIPOC & Jewish people and start showing their asses and they do NOT take constructive feedback well.', 'the more i read european modernists the more i feel that, if i were a progressive jewish person, i would not really want people to read "the old testament"', 'Really disturbing how the status quo position on Jewish people in this country is one given by antisemitic philosemites who only consider Jewish people valid if they support the continued oppression and murder of Palestinians.', 'Explaining Jewish people to twitter "so imagine Ben Gross"', 'it’s so easy to spot an arab woman if she’s telling a story she feels some type of way about, she doesn’t need to have an accent or anything lmaooo i’ll just know that’s my people', 'This is a battle for the soul of White America. The GOP have become full fledged fascists now & the ONLY GROUP they win as a majority is non Jewish White people. None of them would be in power without this group. So are you gonna be fascists or save democracy. Your “choice”', 'Kanye West Vs rat in cheese maze: Kanye west starts rapping about how the rat is jewish. The rat goes through maze and eats the cheese. The rat is undefeated against this endless lineup of foolish people', 'really cool that hackers are selling doxing info for like 1,000,000 Ashkenazi Jewish people thats just damned great. I hate it here.', 'The Chinese educate their kids in Chinese, the Arabs educate their kids in Arabic, the British educate their kids in English. It’s only in Africa education is defined by how much we can copy other people.', 'A lot of non-Jewish people need to get it through their heads that the amount of time Jewish people spend openly talking to non-Jewish people about our Judaism is no indication of the depth of our belief.', 'There are only 16 million Jewish people in the entire world. Y’all gotta start working with us & educate yourselves about antisemitism.'] 

from dataclasses import dataclass

@dataclass
class Post:
    status: str
    text: str
    sentiment: str 
    toxicity: str 

all = []

for post in before: 
    all.append(Post('Before', post, '', ''))
for post in after: 
    all.append(Post('After', post, '', ''))

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Define a function to perform sentiment analysis
def analyze_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probs = torch.softmax(logits, dim=1).detach().numpy()[0]

    # Determine sentiment label
    label_mapping = {
        0: "Very negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very positive"
    }
    sentiment_label = label_mapping[int(probs.argmax())]

    return sentiment_label, probs

negative_sentences = []
positive_sentences = []

for post in all:
    # Perform sentiment analysis
    text = post.text
    sentiment, probabilities = analyze_sentiment(text)
    post.sentiment = sentiment


In [9]:
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline

model_path = "JungleLee/bert-toxic-comment-classification"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# print(pipeline("Whatever happens I love you all and the sun will come up tomorrow. Now lets see those memes!."))

for post in all:
    # Perform sentiment analysis
    text = post.text
    result = pipeline(text)
    for r in result: 
        label, score = r['label'], r['score']
        post.toxicity = label

print(all)



[Post(status='Before', text='Two things I periodically relearn and forget are that there are less Jewish people in the world than you’d think, and that bras are pretty expensive.', sentiment='Negative', toxicity='toxic'), Post(status='Before', text='Whether its Democrats intervening to make sure Jewish people arent targeted on Twitter, or Elon Musk intervening to make sure murderous Russians arent targeted by Ukrainians who want to defend their own country, both sides need to mind their own business.', sentiment='Very negative', toxicity='non-toxic'), Post(status='Before', text='Israeli police are brutally cracking down on Eritrean refugees in Tel Aviv. The land they dispossessed Palestinians from and made them refugees.  Today we see the essence and the core of Israeli supremacy. A single ethnoreligius state for White Jewish people.', sentiment='Very positive', toxicity='toxic'), Post(status='Before', text='not to be middle eastern on the tl but i’m here freezing my ass off in my turt